In [ ]:
# General
import numpy as np
import matplotlib.pyplot as plt
import time

# Linear regression
import sklearn.linear_model

# CNN
import torch
from torch import nn, cuda
from torch.utils.data import DataLoader, Dataset

# KFold
from sklearn.model_selection import KFold

# Data preprocessing

In [ ]:
# Load digits array from file
# File is formatted 200 rows per digit, each row representing a 15x16 grayscale image

digits = np.loadtxt('ProjectDigits_materials/mfeat-pix.txt', usecols=range(240))
digits = digits.reshape(digits.__len__(), 16, 15)

In [ ]:
plt.imshow(digits[0], cmap='gray')

In [ ]:
# Split the data in train and test data, 50/50
train_digits = np.empty([0, 16, 15])
x_test = np.empty([0, 16, 15])
y_test = np.empty(0)

digits_per_digit = np.array(np.split(digits, 10))

for d in range(10):
    [d_train, d_test] = np.array(np.split(digits_per_digit[d], 2))
    print(d_train.shape)
    train_digits = np.concatenate((train_digits, d_train))
    x_test = np.concatenate((x_test, d_test))
    y_test = np.concatenate((y_test, np.full(d_test.__len__(), d)))

In [ ]:
# Train data is augmented by shifting in the 8 cardinal directions
x_train_augmented = np.empty([0, 16, 15])
y_train_augmented = np.empty(0)

train_digits_per_digit = np.array(np.split(train_digits, 10))

for d in range(10):
    train_digits_10_chunks = np.array(np.split(train_digits_per_digit[d], 10))
    train_digits_10_chunks = np.concatenate(train_digits_10_chunks[:10])
    # Perform data augmentation by shifting in 8 cardinal directions
    for dx in [-1, 0, 1]:
        for dy in [-1, 0, 1]:
            if dx == 0 and dy == 0:
                continue  # skip the original data
            shifted_digits = np.roll(train_digits_10_chunks, (dx, dy), axis=(1, 2)) #roll used to shift the data
            x_train_augmented = np.concatenate((x_train_augmented, shifted_digits))
            y_train_augmented = np.concatenate((y_train_augmented, np.full(shifted_digits.shape[0], d)))
            #labels for the augmenteddata created using np.full to fill an array of the same length

In [ ]:
# Normalize the data
x_test = x_test / x_test.max()
x_train_augmented = x_train_augmented / x_train_augmented.max()

# Linear regression
A basic linear regression to sanity check results from CNN

In [ ]:
# Flatten the data
x_train_flat = np.reshape(x_train_augmented, (x_train_augmented.__len__(), -1))
x_test_flat = np.reshape(x_test, (x_test.__len__(), -1))

In [ ]:
# Teach basic linear regression with train data, test with test data
lr = sklearn.linear_model.LogisticRegression(solver='saga', tol=0.1)
lr = lr.fit(x_train_flat, y_train_augmented)

print(f'train: {lr.score(x_train_flat, y_train_augmented)}')
print(f'test: {lr.score(x_test_flat, y_test)}')

# CNN

## Pytorch preprocessing

In [ ]:
device = torch.device('cuda' if cuda.is_available() else 'cpu')

In [ ]:
# Special torch arrays
# Taken from mnist_keras_pytorch.ipynb from tutorial
torch_x_train = torch.from_numpy(x_train_augmented).unsqueeze(1)
torch_y_train = torch.from_numpy(y_train_augmented)

torch_x_test = torch.from_numpy(x_test).unsqueeze(1)
torch_y_test = torch.from_numpy(y_test)

In [ ]:
# Torch datasets
# Taken from mnist_keras_pytorch.ipynb from tutorial
class DigitsDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __len__(self):
        return len(self.Y)

    def __getitem__(self, index):
        X = self.X[index].float()
        Y = self.Y[index].long()
        return X,Y

train_dataset = DigitsDataset(torch_x_train, torch_y_train)

test_dataset = DigitsDataset(torch_x_test, torch_y_test)

In [ ]:
# Torch dataloader
# Taken from mnist_keras_pytorch.ipynb from tutorial
if cuda.is_available():
    loader_args = dict(shuffle=True, batch_size=256, num_workers=8, pin_memory=True)
else:
    loader_args = dict(shuffle=True, batch_size=64)

train_loader = DataLoader(train_dataset, **loader_args)

test_loader = DataLoader(test_dataset, **loader_args)

## Epochs setup

In [ ]:
# Taken from mnist_keras_pytorch.ipynb from tutorial
def train_epoch(model, train_loader, criterion, optimizer, device, do_print=True):
    model.train()

    running_loss = 0.0

    start_time = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()   # .backward() accumulates gradients
        data = data.to(device)
        target = target.to(device) # all data & model on same device

        outputs = model(data)
        loss = criterion(outputs, target)
        running_loss += loss.item()

        loss.backward()
        optimizer.step()

    end_time = time.time()

    running_loss /= len(train_loader)
    if do_print:
        print('Training Loss: ', running_loss, 'Time: ',end_time - start_time, 's')
    return running_loss

In [ ]:

# Taken from mnist_keras_pytorch.ipynb from tutorial
def test_model(model, test_loader, criterion, device, testing_mode=False, do_print=True):
    with torch.no_grad():
        model.eval()

        running_loss = 0.0
        total_predictions = 0.0
        correct_predictions = 0.0

        for batch_idx, (data, target) in enumerate(test_loader):
            data = data.to(device)
            target = target.to(device)
            outputs = model(data)


            _, predicted = torch.max(outputs.data, 1)
            total_predictions += target.size(0)
            correct_predictions += (predicted == target).sum().item()

            loss = criterion(outputs, target).detach()
            running_loss += loss.item()

        running_loss /= len(test_loader)
        acc = (correct_predictions/total_predictions)*100.0
        if do_print:
            mode = 'Testing' if testing_mode else 'Validation'
            print(mode, ' Loss: ', running_loss)
            print(mode, ' Accuracy: ', acc, '%')
        return running_loss, acc

In [ ]:
def train_model(model, train_loader, check_loader, criterion, optimizer, device, epochs=10, testing_mode=False, print_freq=1):
    if print_freq > 0:
        print(model)
    model.to(device)

    Train_loss = []
    Check_loss = []
    Check_acc = []

    for i in range(epochs):
        do_print = print_freq > 0 and (i == 0 or (i+1) % print_freq == 0)
        if do_print:
            print('Epoch : ', i+1)
        train_loss = train_epoch(model, train_loader, criterion, optimizer, device, do_print=do_print)
        check_loss, check_acc = test_model(model, check_loader, criterion, device, testing_mode=testing_mode, do_print=do_print)
        Train_loss.append(train_loss)
        Check_loss.append(check_loss)
        Check_acc.append(check_acc)
        if do_print:
            print('='*20)

    return Train_loss, Check_loss, Check_acc

In [ ]:
def kfold_train(model_type, criterion, optimizer, train_data_x, train_data_y, device, loader_args, num_folds=2, epochs=10, shuffle=True, random_state=42, print_freq=1):
    # Create the KFold object
    kf = KFold(num_folds, shuffle=shuffle, random_state=random_state)

    Fold_train_loss = []
    Fold_val_loss = []
    Fold_val_acc = []

    # Do the folds
    for fold, (train_index, val_index) in enumerate(kf.split(train_data_x)):
        print(f"Fold {fold+1}/{num_folds}")

        # Create datasets
        fold_train_dataset = DigitsDataset(train_data_x[train_index], train_data_y[train_index])
        fold_val_dataset = DigitsDataset(train_data_x[val_index], train_data_y[val_index])

        # Create dataloaders
        fold_train_loader = DataLoader(fold_train_dataset, **loader_args)
        fold_val_loader = DataLoader(fold_val_dataset, **loader_args)

        # Create new model for fold
        fold_model = model_type()

        # Reset the optimizer
        optimizer.param_groups.clear()
        optimizer.add_param_group({'params': fold_model.parameters()})

        # Train the model
        train_loss, val_loss, val_acc = train_model(fold_model, fold_train_loader, fold_val_loader, criterion, optimizer, device, epochs=epochs, print_freq=print_freq)

        # Store metrics
        Fold_train_loss.append(train_loss)
        Fold_val_loss.append(val_loss)
        Fold_val_acc.append(val_acc)

    return Fold_train_loss, Fold_val_loss, Fold_val_acc

## Network attempts
https://madebyollin.github.io/convnet-calculator/ can come in very handy when determining the trainable parameters for a convolution layer.

### LeNet-5-like network (Tom)
The LeNet-5 architecture was the first successfully trained CNN - it was even designed for the task we're doing, but on images of 28x28 (hence the "-like").

Its architecture is as follows:
1. Input (1@15x16)
2. Convolution -- (6@15x16)
3. Average pool -- (6@7x8)
4. Convolution -- (16@5x6)
5. Average pool -- (16@2x3)
6. Fully connected layer -- 120 nodes
7. Fully connected layer -- 84 nodes
8. Fully connected output -- one-hot encoding

In [ ]:
leNet5 = nn.Sequential(
    nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding=2), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Conv2d(in_channels=6, out_channels=16, kernel_size=3), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(in_features=96, out_features=120), nn.Sigmoid(),
    nn.Linear(in_features=120, out_features=84), nn.Sigmoid(),
    nn.Linear(in_features=84, out_features=10)
)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(leNet5.parameters())

# train_loss, test_loss, test_acc = train_model(leNet5, train_loader, test_loader, criterion, optimizer, 20, print_freq=0)
# plt.plot(test_acc)

### Tom's 2nd attempt

Architecture as follows:
1. Input (1@15x16)
2. Convolution (1@16x16)
3. Convolution (6@16x16)
4. Average pool (6@8x8)
5. FCL 240
6. FCL 60
7. FCL Output -- one-hot encoded

In [ ]:
class Tom2(torch.nn.Module):
    def __init__(self):
        super(Tom2, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=(5,4), padding=2)
        self.activ1 = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, padding=2)
        self.activ2 = nn.ReLU()
        self.avg3 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.flat4 = nn.Flatten()
        self.lin5 = nn.Linear(in_features=384, out_features=240)
        self.activ5 = nn.ReLU()
        self.lin6 = nn.Linear(in_features=240, out_features=60)
        self.activ6 = nn.ReLU()
        self.lin7 = nn.Linear(in_features=60, out_features=10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.activ1(x)
        x = self.conv2(x)
        x = self.activ2(x)
        x = self.avg3(x)
        x = self.flat4(x)
        x = self.lin5(x)
        x = self.activ5(x)
        x = self.lin6(x)
        x = self.activ6(x)
        x = self.lin7(x)
        return x

tom2_criterion = nn.CrossEntropyLoss()
tom2_optimizer = torch.optim.SGD(Tom2().parameters(), lr=0.01, momentum=0.9)

In [ ]:
k_folds = 2
fold_train_loss, fold_val_loss, fold_val_acc = kfold_train(Tom2, tom2_criterion, tom2_optimizer, torch_x_train, torch_y_train, device, loader_args, num_folds=k_folds, epochs=15, print_freq=5)

In [ ]:
# Plot the training and validation loss for each fold
plt.figure(figsize=(10, 5))
for fold in range(k_folds):
    plt.plot(fold_train_loss[fold], label=f"Fold {fold+1} Train Loss")
    plt.plot(fold_val_loss[fold], label=f"Fold {fold+1} Validation Loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

# Plot the validation accuracy for each fold
plt.figure(figsize=(10, 5))
for fold in range(k_folds):
    plt.plot(fold_val_acc[fold], label=f"Fold {fold+1} Validation Accuracy")
plt.xlabel('Epochs')
plt.ylabel('Accuracy (%)')
plt.title('Validation Accuracy')
plt.legend()
plt.show()

In [ ]:
#reset the model
tom2 = Tom2()
tom2_optimizer.param_groups.clear()
tom2_optimizer.add_param_group({'params': tom2.parameters()})
train_loss, test_loss, test_acc = train_model(tom2, train_loader, test_loader, tom2_criterion, tom2_optimizer, device, epochs=15, testing_mode=True, print_freq=5)

print(test_acc[-1])